In [1]:
import requests
import pandas as pd
import json
import os
import re
import time
import seaborn as sns
import matplotlib.pyplot as plt
from chembl_webresource_client.new_client import new_client  # optional

from rdkit import Chem
from rdkit.Chem import PandasTools
from rdkit.Chem.Draw import IPythonConsole

In [6]:
headers = {"x-api-key": "YOUR_MOLPORT_API_KEY"}

In [ ]:
url = "https://api.molport.com/api/v1/catalogues/list"  # example endpoint
res = requests.get(url, headers=headers)
catalogs = res.json().get("catalogues", [])  # adjust based on API spec
df_catalog = pd.DataFrame(catalogs)

In [ ]:
df_catalog.head()